In [1]:
import json
import requests
import time
import logging
import io
import pandas as pd

In [2]:
logging.basicConfig(level=logging.INFO, 
                    format='%(asctime)s %(name)s %(levelname)s:%(message)s')
logger = logging.getLogger(__name__)

In [3]:
class Vacancy():
    def __init__(self, area=113):
        self.area = area
        
    def get_page(self, work_name, prof, exp=None, page=0):
        
        params = {
            'text': work_name,
            'area': self.area,
            'specialization': prof,
            'page': page,
            'per_page': 100  # default 100 (100 vacancy per each page)
        }
        
        if exp:
            params['experience'] = exp
        
        try:
            req = requests.get('https://api.hh.ru/vacancies', params)
            print(req.url)
            list_of_vacancies = req.content.decode()
            req.close()
        except requests.exceptions.RequestException as e:  # This is the correct syntax
            raise SystemExit(e)

        return json.loads(list_of_vacancies)
    
    
    def pars_vacansies():
        path = './data/vacancies.csv'
        f = io.open(path, mode='a', encoding='utf-8')
        j = 391
        vacancy = Vacancy()

        for index, row in jobs.iloc[506:].iterrows():
            job_id = np.round(row.id, 3)
            param = vacancy.get_page('', job_id) # получаем кол-тво вакансий
            found = param['found']

            logger.info(f'{row.job_name} - номер {j} - вакансий {found}')

            if found > 2000: # если вакансий больше 2000
                for exp in experience: # перебираем парамметры опыта работы
                    pages = vacancy.get_page('', job_id, exp['id'])['pages']
                    for page in range(0, pages): # перебираем страницы
                        logger.info(f'{row.job_name} - номер {j} - опыт {exp} - страница {page}')

                        vacancies = vacancy.get_page('', job_id, exp['id'], page)

                        for v in vacancies['items']: # перебираем каждую ваканси.
                            if v.get('snippet'):
                                requirement = str(v['snippet'].get('requirement', 'NaN')).replace(';',' ')
                                responsibility = str(v['snippet'].get('responsibility', 'NaN')).replace(';',' ')
                            else:
                                requirement = 'NaN'
                                responsibility = 'NaN'
                            # заносим вакансию в файл
                            vacancy_id = v.get('id')
                            write_vacancy = f'{vacancy_id};{requirement};{responsibility};{job_id};{row.job_name};{row.specialization}\n'
                            f.write(write_vacancy)

            else:
                for page in range(0, param['pages']):
                    logger.info(f'{row.job_name} - номер {j} - страница {page}')

                    vacancies = vacancy.get_page('', job_id, None, page)

                    for v in vacancies['items']: # перебираем каждую ваканси.
                        if v.get('snippet'):
                            requirement = str(v['snippet'].get('requirement', 'NaN')).replace(';',' ')
                            responsibility = str(v['snippet'].get('responsibility', 'NaN')).replace(';',' ')
                        else:
                            requirement = 'NaN'
                            responsibility = 'NaN'
                        # заносим вакансию в файл
                        vacancy_id = v.get('id')
                        write_vacancy = f'{vacancy_id};{requirement};{responsibility};{job_id};{row.job_name};{row.specialization}\n'
                        f.write(write_vacancy)

            j += 1

        f.close()
        

In [4]:
experience = [{"id":"noExperience","name":"Нет опыта"},{"id":"between1And3","name":"От 1 года до 3 лет"},{"id":"between3And6","name":"От 3 до 6 лет"},{"id":"moreThan6","name":"Более 6 лет"}]

In [5]:
jobs = pd.read_csv('data/hh_jobs.csv')

In [7]:
jobs.iloc[506:]

,id,job_name,specialization
506,24.378,Тренерский состав,"Спортивные клубы, фитнес, салоны красоты"
507,24.379,Администрация,"Спортивные клубы, фитнес, салоны красоты"
508,24.380,Продажи,"Спортивные клубы, фитнес, салоны красоты"
509,29.495,Грузчик,Рабочий персонал
510,29.510,Слесарь,Рабочий персонал
...,...,...,...
616,6.247,Развитие персонала,"Управление персоналом, тренинги"
617,6.254,Рекрутмент,"Управление персоналом, тренинги"
618,6.309,Тренинги,"Управление персоналом, тренинги"
619,6.319,Управление персоналом,"Управление персоналом, тренинги"


In [8]:
# 390

2021-05-25 13:42:08,493 __main__ INFO:Тренерский состав - номер 391 - вакансий 1940
2021-05-25 13:42:08,494 __main__ INFO:Тренерский состав - номер 391 - страница 0


https://api.hh.ru/vacancies?text=&area=113&specialization=24.378&page=0&per_page=100


2021-05-25 13:42:08,802 __main__ INFO:Тренерский состав - номер 391 - страница 1


https://api.hh.ru/vacancies?text=&area=113&specialization=24.378&page=0&per_page=100


2021-05-25 13:42:09,600 __main__ INFO:Тренерский состав - номер 391 - страница 2


...

https://api.hh.ru/vacancies?text=&area=113&specialization=6.336&page=16&per_page=100&experience=between3And6
https://api.hh.ru/vacancies?text=&area=113&specialization=6.336&page=17&per_page=100&experience=between3And6


2021-05-25 13:59:11,772 __main__ INFO:Учет кадров - номер 505 - опыт {'id': 'moreThan6', 'name': 'Более 6 лет'} - страница 0


https://api.hh.ru/vacancies?text=&area=113&specialization=6.336&page=0&per_page=100&experience=moreThan6


2021-05-25 13:59:12,218 __main__ INFO:Учет кадров - номер 505 - опыт {'id': 'moreThan6', 'name': 'Более 6 лет'} - страница 1


https://api.hh.ru/vacancies?text=&area=113&specialization=6.336&page=0&per_page=100&experience=moreThan6
https://api.hh.ru/vacancies?text=&area=113&specialization=6.336&page=1&per_page=100&experience=moreThan6
